In [1]:
import sys
sys.path.append('..')

import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
import time
import faiss
import xarray as xr
from geopy.distance import geodesic
import geopandas as gpd
from sklearn.preprocessing import StandardScaler
from shapely.geometry import Point
from sklearn.impute import SimpleImputer, KNNImputer
from MeteoDataset import MeteoDataset
from scipy.spatial import distance
from load_meteo_data import create_meteo_dataframe
from sklearn.neighbors import BallTree
from scipy.spatial.distance import cdist
from pathlib import Path
from constants import *
from load_era5_data import create_era5_dataframe
from eda_utils import impute_data
import arrow

In [3]:
files = {'Bihar_536_Sensor_Data_Sep_2023_Screened.csv': 7, 'Bihar_536_Sensor_Data_Oct_2023_Screened.csv': 7,\
        'Bihar_536_Sensor_Data_Nov_2023_Screened.csv': 6, 'Bihar_536_Sensor_Data_Jan_2024_Screened.csv': 6,\
        'Bihar_536_Sensor_Data_Dec_2023_Screened.csv': 6, 'Bihar_512_Sensor_Data_May_Aug_Screened_Hourly.csv': 6}

nc_files = [f'{data_bihar}/era5_may_dec_2023.nc', f'{data_bihar}/era5_jan_2024.nc']
locations_fp = f'{data_bihar}/locations.txt'

geojson_file = f'{data_bihar}/bihar.json'
pbl_file = f'{data_bihar}/PBLH_may_Dec_2023.nc'
other_params_file = f'{data_bihar}/Era5_data_May_Dec_2023.nc'
output_meteo_file = f'{data_bihar}/bihar_meteo_may_jan.pkl'
output_meteo_era5_file = f'{data_bihar}/bihar_meteo_era5_may_jan.pkl'
output_meteo_era5_imputed_file = f'{data_bihar}/bihar_meteo_era5_may_jan_iterative_imputed.pkl'
pm25_clustering_file = f'{data_bihar}/bihar_clustering_pm25.csv'
knowair_fp = f'{data_bihar}/KnowAir.npy'

region = gpd.read_file(geojson_file)
# meteo_df = pd.read_pickle(meteo_file)

start_date, end_date = pd.Timestamp('2023-05-01 00:00:00'), pd.Timestamp('2024-02-01 00:00:00')

In [4]:
if Path(output_meteo_file).is_file():
        meteo_df = pd.read_pickle(output_meteo_file)
else:
    meteo_df = create_meteo_dataframe(files, output_meteo_file, start_date, end_date)

In [5]:
min_lon, min_lat, max_lon, max_lat = region.total_bounds
min_lon, min_lat, max_lon, max_lat

(83.320238, 24.286327, 88.29954611201047, 27.521347)

In [6]:
region.total_bounds

array([83.320238  , 24.286327  , 88.29954611, 27.521347  ])

In [7]:
if Path(output_meteo_era5_file).is_file():
        era5_df = pd.read_pickle(output_meteo_era5_file)
else:
    era5_df = create_era5_dataframe(nc_files, meteo_df, output_meteo_era5_file)

In [4]:
if Path(output_meteo_era5_imputed_file).is_file():
    imputed_df = pd.read_pickle(output_meteo_era5_imputed_file)
else:
    imputed_df = impute_data(era5_df, output_meteo_era5_imputed_file, method='iterative')

In [9]:
imputed_df.count()

timestamp    3384864
block        3384864
district     3384864
latitude     3384864
longitude    3384864
rh           3384864
temp         3384864
blh          3384864
u10          3384864
v10          3384864
kx           3384864
sp           3384864
tp           3384864
pm25         3384864
dtype: int64

In [10]:
# df_g = imputed_df.groupby(['latitude', 'longitude'])

# for loc, group in df_g:
#     print(loc, group.shape)

In [11]:
era5_df.count()

index        3384864
timestamp    3384864
block        3384864
district     3384864
latitude     3384864
longitude    3384864
rh           2040291
temp         1907280
blh          3384864
u10          3384864
v10          3384864
kx           3384864
sp           3384864
tp           3384864
pm25         2045544
dtype: int64

In [12]:
# pm25_df = pd.read_csv(pm25_clustering_file)
# pm25_df

In [13]:
# df_g = pm25_df.groupby(['latitude', 'longitude'])

# for loc, group in df_g:
#     print(loc, group.shape)

In [14]:
imputed_df.shape

(3384864, 14)

In [15]:
# imputed_df[imputed_df['pm25'] > imputed_df['pm25'].max()-10]

In [22]:
mu, sigma = imputed_df['pm25'].mean(), imputed_df['pm25'].std()
mu, sigma

(105.63217922057937, 98.01558999664988)

In [3]:
d = np.load(knowair_fp)
d.shape

(11688, 184, 18)

In [4]:
d[0][0]

array([ 1.82038193e+00, -1.31661860e+00,  2.53094174e+02,  2.66073469e+02,
        3.84515500e+01, -4.27048668e+01,  2.09199109e+01,  1.92479888e+01,
        4.33315233e-04,  1.02294539e+05,  2.63243316e+02,  2.64921798e+02,
        0.00000000e+00,  7.02799650e+00, -4.34368872e+00,  6.31098542e-01,
       -6.50030234e-05,  0.00000000e+00])

In [14]:
locs = list(imputed_df.groupby(['block', 'district', 'longitude', 'latitude']).groups.keys())

loc_df = pd.DataFrame(data=locs)
loc_df.to_csv(locations_fp, sep='|', header=False, index=True)